# WS_follow_along_nanogpt.ipynb
# WESmith 06/10/23
## follow along with Karpathy video
## https://www.youtube.com/watch?v=kCc8FmEb1nY

In [ ]:
import torch
import torch.nn as nn
import numpy    as np
from   torch.nn import functional as F

In [ ]:
# hyperparameters
# apparently block_size is somehow coupled to other params to give best results:
block_size    = 8    # number of context characters (original was 8, 12 gave garbage, 16 not as good as 8)
n_embd        = 32   # size of embedding space
head_size     = 16   # head size for self-attention
n_head        = 4    # number of heads
n_layer       = 3    # number of layers
dropout       = 0.2

batch_size    = 32
max_iters     = 5000
eval_interval = 500
learning_rate = 1e-3
eval_iters    = 200  # used in estimate_loss() function

seed          = 1337
device        = 'cpu'

In [ ]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
len(text)

In [ ]:
text[:150]

In [ ]:
print(text[:150])

In [ ]:
''.join(list(set(text)))

In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
''.join(chars)

In [ ]:
print(vocab_size)

In [ ]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [ ]:
decode([30])

In [ ]:
print(encode('hii there,\nyou'))
print(decode(encode('hii there,\nyou')))

In [ ]:
data = torch.tensor(encode(text), dtype=torch.long)

In [ ]:
data.shape, data.dtype

In [ ]:
print(data[:100])

In [ ]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data   = data[n:]

In [ ]:
train_data.shape, val_data.shape

In [ ]:
decode(train_data[:block_size + 1].numpy())  # WS grab the array with .numpy()

In [ ]:
train_data[1]

In [ ]:
train_data[1].tolist()

In [ ]:
x = train_data[:block_size]       # context data
y = train_data[1:block_size + 1]  # target data

In [ ]:
x, y

In [ ]:
for t in range(block_size):
    context = x[:t + 1].tolist()  # WS added tolist() for decode()
    target  = y[t].tolist()
    print(f'when input is: {decode(context)} the target is: {decode([target])}')  # need to put target value into a list for decode()

In [ ]:
torch.manual_seed(seed)

In [ ]:
def get_batch(batch_size, split, block_size=8):  # WS added batch_size, block_size as args
    data = train_data if split=='train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x  = torch.stack([data[i:i + block_size] for i in ix])
    y  = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
    return x, y

@torch.no_grad()
def estimate_loss(model, batch_size):  # WS added model, batch_size as args, instead of relying on global
    out = {}
    model.eval() # set model to eval phase (check docs for meaning of this)
    # presumably eval() and train() methods are in nn.Module parent class of model
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(batch_size, split)
            _, loss = model(X, Y)
            losses[k]    = loss.item()
        out[split] = losses.mean()
    model.train() # reset model back to training phase (check docs for meaning)
    return out

In [ ]:
test_size = 4
xb, yb = get_batch(test_size, 'train')
print('inputs')
print(xb.shape)
print(xb)
print('targets')
print(yb.shape)
print(yb)

In [ ]:
for b in range(test_size):
    for t in range(block_size):
        context = xb[b, :t + 1]
        target  = yb[b, t]
        print(f'when input is: {decode(context.tolist())}| the target is: {decode([target.tolist()])}')

## REVIEW OF HOW PyTorch EMBEDDING WORKS

In [ ]:
# WS a pytorch embedding is just a random mxn array
dd = nn.Embedding(7,4)  # vocab_size x embedding_size (C for channel), 7 x 4
ee = list(dd.parameters())  # parameters() is a generator, this returns a one-element list
ee[0].shape
ee[0].detach().numpy() # this is how to access the array

In [ ]:
# review of how embedding works: not a matrix multiply: each x value pulls out a row of the embedding table
ff = torch.tensor([[2,1,5],[1,2,1]])  # batch (B) x num_chars (time or T), 2 x 3
gg = dd(ff)  # batch x num_chars x C, 2 x 3 x 4: 
# thus embedding is (vocab_size, C), x is (B,T), output is (B,T,C): as if each scalar at B,T is replaced with a 1D array C long
# vocab_size is lost, but the values of x must be in the range of vocab_size (ie, the values of x must be in the vocabulary)
ff, gg

In [ ]:
class Head(nn.Module):
    '''one head of self-attention'''
    
    def __init__(self, head_size, n_embd, block_size, dropout):  # WS added n_embd, block_size args
        super().__init__()
        self.head_size = head_size # WS added
        self.key    = nn.Linear(n_embd, head_size, bias=False)
        self.query  = nn.Linear(n_embd, head_size, bias=False)
        self.value  = nn.Linear(n_embd, head_size, bias=False)  # WS! should this be (n_embd, n_embd)? see comment below
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        B,T,C = x.shape   # C is n_embd
        k = self.key(x)   # (B,T,C) @ (C,head_size) => (B,T,head_size)
        q = self.query(x) # (B,T,head_size)
        
        # compute attention scores ('affinities')
        # WS comment: is the following line incorrect? supposed to be normalizing by head_size, not C = n_embd
        #wei = q @ k.transpose(-2,-1) * C**-0.5 # (B,T,head_size) @ (B,head_size,T) => (B,T,T)
        wei = q @ k.transpose(-2,-1) * self.head_size**-0.5  # WS mod
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        #print(f'Head: k.shape {k.shape} q.shape {q.shape} wei.shape {wei.shape}')
        wei = F.softmax(wei, dim=-1) # (B,T,T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C) @ (C,head_size) => (B,T,head_size)
        out = wei @ v # (B,T,T) @ (B,T,head_size) => (B,T,head_size) # WS! this is a problem: it is returning head_size instead of n_embd
        #print(f'Head: out.shape {out.shape}')
        return out


In [ ]:
class MultiHeadAttention(nn.Module):
    '''multiple heads of self-attention in parallel'''
    
    def __init__(self, num_heads, head_size, n_embd, block_size, dropout):  # WS added args
        super().__init__()
        self.heads   = nn.ModuleList([Head(head_size, n_embd, block_size, dropout) for _ in range(num_heads)])
        self.proj    = nn.Linear(n_embd, n_embd)  # not clear why added: added at 1h 31m in video
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # concatenate over channel dim (head_size)
        out = self.dropout(self.proj(out))
        return out

In [ ]:
class FeedForward(nn.Module):
    '''a simple linear layer followed by a nonlinearity'''
    
    def __init__(self, n_embd, dropout):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd), # factor of 4 from 2017 paper: black magic
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),   # this is the 'proj' network
            nn.Dropout(dropout), )
        
    def forward(self, x):
        return self.net(x)

In [ ]:
class Block(nn.Module):
    '''Transformer block: communication followed by computation'''
    
    def __init__(self, n_embd, n_head, block_size, dropout):  # WS added arg
        # n_embd: embedding dim, n_head: number of heads
        super().__init__()
        head_size = n_embd // n_head  # WS guarantees (assuming no roundoff error) that final concatenated output has size n_embd
        self.sa   = MultiHeadAttention(n_head, head_size, n_embd, block_size, dropout)  # 'sa' for self-attention
        self.ffwd = FeedForward(n_embd, dropout)
        self.ln1  = nn.LayerNorm(n_embd)  # layernorm discussion starts around 1h 33m in video: slight departure from 2017 paper
        self.ln2  = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x)) # 'skip' approach
        x = x + self.ffwd(self.ln2(x))
        return x

In [ ]:
# bigram model seed
torch.manual_seed(seed)

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size, block_size, n_embd, head_size, n_head, n_layer, dropout):  # WS added args
        super().__init__()
        # create a random embedding matrix
        self.token_embedding_table    = nn.Embedding(vocab_size, n_embd) # size (vocab, C) C is embedding size n_embd
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        
        # !WS NOTE: head_size set to n_embd here, may change later: if head_size != n_embd code crashes in the current form
        # this comment made in the video at 1h 20m 25s in video: 'keeping head size as n_embd just for now'
        #self.sa_head                  = Head(n_embd, n_embd, block_size)
        
        # since each head size is 8, and 4 heads will be concatenated, the final channel size is 32, which matches n_embd
        # if n_embd size isn't matched by head_size x num_heads, an error will result in lm_head() matrix multiply with existing code
        #self.sa_heads = MultiHeadAttention(4, n_embd//4, n_embd, block_size) # 4 heads, head_size = 8 for n_embd = 32
        #self.ffwd = FeedForward(n_embd)
        
        '''
        self.blocks = nn.Sequential(
            Block(n_embd, n_head=4, block_size=block_size),
            Block(n_embd, n_head=4, block_size=block_size),
            Block(n_embd, n_head=4, block_size=block_size),
            nn.LayerNorm(n_embd))
        '''
        # the '*' indicates a list is input
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head, block_size=block_size, dropout=dropout) for _ in range(n_layer)])
        self.ln_f   = nn.LayerNorm(n_embd)  # final layer norm
        
        # lm_head means 'language-modeling head'
        # WS note: to be most general, the input dimension of lm_head must be head_number * head_size, since this is the
        #          final size of the concatenated multihead output
        self.lm_head    = nn.Linear(n_embd, vocab_size)  # this introduced in video at 59m
        
        #self.vocab_size = vocab_size  # WS
        self.block_size = block_size  # WS
        #self.n_embd     = n_embd      # WS
        #self.head_size  = head_size   # WS
        
    def forward(self, idx, targets=None):
        # B = batch, T = time, C = channel (embedding dimension size n_embd)
        # idx and targets are each (B,T) tensors of integers
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx) # size (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C) due to broadcasting of pos_emb over B
        #x = self.sa_head(x) # apply one head of self-attention, (B,T,C)
        #x = self.sa_heads(x) # multiple heads
        #x = self.ffwd(x) # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x)   # (B,T,C)
        #print(f'x.shape = {x.shape}') #', lm_head.shape = {self.lm_head.shape}')
        logits  = self.lm_head(x) # (B,T,C) @ (C,vocab_zsize) => (B,T,vocab_size)
        
        if targets is None:
            loss = None
        else:
            # this is training the embedding matrix
            B, T, C = logits.shape
            logits  = logits.view(B * T, C)  # reshape for cross_entropy
            targets = targets.view(B * T)    # ditto (or could do ...view(-1))
            loss    = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    # this function is more general than a digraph model (block_size = 1) requires, 
    # for use later with longer values pf block_size
    def generate(self, idx, max_new_tokens):
        # idx is a (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -self.block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # this becomes (B, C)
            # apply softmax to get probs
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T + 1)
        return idx
        

In [ ]:
# build the model, get the data
xb, yb = get_batch(batch_size, 'train')
model = BigramLanguageModel(vocab_size, block_size, n_embd, head_size, n_head, n_layer, dropout)
m     = model.to(device)
out, loss = m(xb, yb)
out.shape, loss.item()

In [ ]:
batch = 1  # WS mod
idx = torch.zeros((batch, 1), dtype=torch.long)
out = m.generate(idx, max_new_tokens=500)
#print(out.shape)
for k in out:
    print(decode(k.tolist()))

In [ ]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# training
for iter in range(max_iters + 1):  # WS added 1 to get last result to print
    
    if iter % eval_interval == 0:
        losses = estimate_loss(model, batch_size)
        print(f"step {iter:5}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    
    xb, yb = get_batch(batch_size, 'train')
    
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

In [ ]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))

# The mathematical trick in self-attention

In [ ]:
# setup
torch.manual_seed(seed)
#B,T,C = 4, 8, 32 # batch, time, channels (encoding size): video values
B,T,C = 2, 3, 4 # WS values
#x = torch.randn(B, T, C) # video values
x = torch.reshape(torch.arange(0, B*T*C).float(), (B,T,C))  # WS
x

In [ ]:
# version 1: explicit calculation of progressive means of rows
# we want x[b, t] = mean_{i<=t} x[b, i]
# ie, calculate the average of what comes before the ith token
# and include the ith token in the average
xbow = torch.zeros((B, T, C)) # bow = 'bag of words'
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (T, C)
        xbow[b, t] = torch.mean(xprev, 0) # (C)
xbow  # first row is preserved, second row is average of first two rows, third row is average of first three rows

In [ ]:
# version 2: use triangle matrix
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (T,T) @ (B,T,C) => (B,T,C) (wei is broadcast over B)
torch.allclose(xbow, xbow2)

In [ ]:
wei

In [ ]:
# version 3: use softmax
tril = torch.tril(torch.ones(T, T))
wei  = torch.zeros((T, T))
wei  = wei.masked_fill(tril == 0, float('-inf'))
wei  = F.softmax(wei, dim=-1)  # normalize along last dimension
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

In [ ]:
# version 4: self attention!
torch.manual_seed(seed)
#B,T,C = 4,8,32 # batch, time, channels, video values
B,T,C = 2,3,4 # WS values
x = torch.randn(B, T, C)

# a single Head for self-attention
#head_size = 16 # video value
head_size = 5 # WS value
key   = nn.Linear(C, head_size, bias=False) # (C,head)
query = nn.Linear(C, head_size, bias=False) # (C, head)
value = nn.Linear(C, head_size, bias=False) # (C. head)
k     = key(x)   # what the token CONTAINS, (B,T,C) @ (C,head) => (B,T,head): for every batch, token, there is a head vector of info about the token
q     = query(x) # what the token is INTERESTED IN, (B,T,head)
v     = value(x) # raw x is PRIVATE info about token (the original embedding): v is the info TO BE COMMUNICATED by the token (B,T,head)
# self-attention means: the key and values both come from the same x
# here's the magic to get the weights:
# multiply by head_size**-5 to normalize the variance of the weights
wei = q @ k.transpose(-2,-1) * head_size**-0.5 # (B,T,head) @ (B,head,T) => (B,T,T); transpose last two dimensions of k

In [ ]:
wei

In [ ]:
tril = torch.tril(torch.ones(T, T))
# NOTE: in decoder methods, use the causal line below (ie, the future not known)
#       in coder   methods, don't use the line below: all nodes can communicate
wei  = wei.masked_fill(tril == 0, float('-inf'))
wei  = F.softmax(wei, dim=-1)
outx  = wei @ x
outv  = wei @ v

In [ ]:
wei, print(), x, print(), outx, print(), outv

In [ ]:
# demonstration of normalizing with head_size
BB,TT,hhead_size = 4,8,16 # larger numbers for significance
k = torch.randn(BB,TT,hhead_size)
q = torch.randn(BB,TT,hhead_size)
wei = q @ k.transpose(-2,-1) * hhead_size**-0.5
k.var(), q.var(), wei.var()